<a href="https://colab.research.google.com/github/it21331022/FruitIdentifier/blob/Riz/densenetfruit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from skimage import io, color, transform
import joblib
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Set the path to the dataset
train_data_path = '/kaggle/input/fruits-fresh-and-rotten-for-classification/dataset/train/'
test_data_path = '/kaggle/input/fruits-fresh-and-rotten-for-classification/dataset/test'

# Create empty lists to store data and labels
data = []
labels = []

def load_and_preprocess_images(folder_path, label):
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            image = io.imread(os.path.join(folder_path, filename))
            # Convert grayscale to RGB by duplicating the single channel
            if image.shape[2] == 1:
                image = np.stack((image,) * 3, axis=-1)
            # Preprocess the image
            image = transform.resize(image, (100, 100, 3))
            data.append(image)
            labels.append(label)

# Load and preprocess images from the "train" folder
load_and_preprocess_images(os.path.join(train_data_path, 'freshapples'), 'freshapples')
load_and_preprocess_images(os.path.join(train_data_path, 'freshbanana'), 'freshbanana')
load_and_preprocess_images(os.path.join(train_data_path, 'freshoranges'), 'freshoranges')
load_and_preprocess_images(os.path.join(train_data_path, 'rottenapples'), 'rottenapples')
load_and_preprocess_images(os.path.join(train_data_path, 'rottenbanana'), 'rottenbanana')
load_and_preprocess_images(os.path.join(train_data_path, 'rottenoranges'), 'rottenoranges')
# Convert data and labels to NumPy arrays
data = np.array(data, dtype=np.float32)  # Ensuring data in float32
labels = np.array(labels)

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(data, labels, test_size=0.2, random_state=42)

class CustomDataset(Dataset):
    def __init__(self, data, labels, label_mapping, transform=None):
        self.data = data
        self.labels = labels
        self.label_mapping = label_mapping
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        numerical_label = self.label_mapping[label]  # Convert the string label to an integer label

        # Ensure the image has the correct dimensions (HxWxC)
        if image.ndim == 2:  # If it's a grayscale image
            image = np.expand_dims(image, axis=2)

        # Convert the data type to uint8
        image = (image * 255).astype(np.uint8)

        # Apply the transformation
        if self.transform:
            image = self.transform(image)

        sample = {
            'image': image,
            'label': numerical_label
        }

        return sample

# Define separate transformations
to_pil = transforms.ToPILImage()
to_tensor = transforms.ToTensor()

# Define a label mapping for your classes
label_mapping = {
    'freshapples': 0,
    'freshbanana': 1,
    'freshoranges': 2,
    'rottenapples': 3,
    'rottenbanana': 4,
    'rottenoranges': 5
}

# Create instances of CustomDataset for training and validation data, providing the label mapping
train_data = CustomDataset(data=X_train, labels=y_train, label_mapping=label_mapping, transform=transforms.Compose([to_pil, to_tensor]))
valid_data = CustomDataset(data=X_valid, labels=y_valid, label_mapping=label_mapping, transform=transforms.Compose([to_pil, to_tensor]))



# Create data loaders
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)

# Define the DenseNet model
model = torchvision.models.densenet121(pretrained=False)

# Freeze all layers (optional, can fine-tune if needed)
for param in model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to match the number of classes
num_classes = 6  # Assuming you have 6 classes (fruits categories)
model.classifier = nn.Linear(model.classifier.in_features, num_classes)

# Define a loss function and an optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the DenseNet model
num_epochs = 20  #youcan adjust this
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in train_loader:
        inputs = batch['image'].to(device)
        labels = torch.Tensor(batch['label']).long().to(device)  # Convert labels to tensor
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

# Evaluate accuracy on the validation dataset
def evaluate_accuracy(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:
            inputs = batch['image'].to(device)
            labels = batch['label'].to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

validation_accuracy = evaluate_accuracy(model, valid_loader, device)
print(f'Validation Accuracy: {validation_accuracy * 100:.2f}%')

# Save the trained model using pickle
model_weights = model.state_dict()

# Specify the filename for the pickle file
pickle_filename = 'densenet_fruit_classifier.pkl'

with open(pickle_filename, 'wb') as file:
    pickle.dump(model_weights, file)




Epoch 1, Loss: 1.1225975468045188
Epoch 2, Loss: 0.7743470596743154
Epoch 3, Loss: 0.6769833641174512
Epoch 4, Loss: 0.622593594339741
Epoch 5, Loss: 0.5803763925374209
Epoch 6, Loss: 0.5617445294673626
Epoch 7, Loss: 0.5278311181308586
Epoch 8, Loss: 0.5158971274073744
Epoch 9, Loss: 0.50339362516508
Epoch 10, Loss: 0.4830577837460207
Epoch 11, Loss: 0.4779309459003337
Epoch 12, Loss: 0.47747741028284413
Epoch 13, Loss: 0.4527259096230343
Epoch 14, Loss: 0.43742546608378163
Epoch 15, Loss: 0.4499295664466781
Epoch 16, Loss: 0.4465513281223975
Epoch 17, Loss: 0.4294919482408426
Epoch 18, Loss: 0.4222093966308531
Epoch 19, Loss: 0.4099957598543866
Epoch 20, Loss: 0.4165265725536661
Validation Accuracy: 86.52%


In [ ]:
import numpy as np
from skimage import io, color, transform
import torch
from torchvision import transforms as torch_transforms
import torch.nn as nn
import joblib

# Load the trained DenseNet model
model = torchvision.models.densenet121(pretrained=False)
num_classes = 6  # Assuming you have 6 classes (fruits categories)
model.classifier = nn.Linear(model.classifier.in_features, num_classes)
model.load_state_dict(torch.load('densenet_fruit_classifier.h5'))
model.eval()

# Define a function to load and preprocess images
def load_and_preprocess_image(image_path):
    image = io.imread(image_path)
    # Convert RGBA to RGB format if needed
    if image.shape[2] == 1:
        image = np.stack((image,) * 3, axis=-1)
    # Preprocess the image - resize and normalize
    image = transform.resize(image, (100, 100, 3))
    image = (image * 255).astype(np.uint8)

    # Create a transform to match the preprocessing used in training
    to_tensor = torch_transforms.ToTensor()
    normalize = torch_transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    preprocessing = torch_transforms.Compose([to_tensor, normalize])

    image = preprocessing(image)
    return image

# Path to the new image you want to classify
new_image_path = '/kaggle/input/fruits-fresh-and-rotten-for-classification/dataset/test/freshapples/Screen Shot 2018-06-08 at 5.09.31 PM.png'

# Preprocess the new image
image_tensor = load_and_preprocess_image(new_image_path)

# Use the DenseNet model to make predictions
with torch.no_grad():
    output = model(image_tensor.unsqueeze(0))  # Pass a batch of size 1

predicted_class = torch.argmax(output).item()

# Map the numeric class label to the corresponding fruit category
class_to_category = {0: 'freshapples', 1: 'freshbanana', 2: 'freshoranges', 3: 'rottenapples', 4: 'rottenbanana', 5: 'rottenoranges'}
predicted_fruit = class_to_category[predicted_class]

print(f"The image is classified as: {predicted_fruit}")


The image is classified as: rottenapples
